<a href="https://colab.research.google.com/github/VasylDvorakDS/AI-Tutor-of-English-by-Open-AI/blob/master/AI_Tutor_of_English_by_Open_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Устанавливаем и импортируем библиотеки, получаем ключ API OpenAI, подгружаем БД для диалога

# Устанавливаем необходимые библиотеки с помощью pip
!pip install faiss-cpu langchain==0.0.271 openai==0.28.1 tiktoken  # Установка версий библиотек faiss-cpu, langchain, openai и tiktoken
#!pip install langchain tiktoken  # Установка библиотек langchain и tiktoken (может быть повторно для подтверждения актуальности версий)

# Импортируем необходимые модули и библиотеки
import os  # Модуль для работы с операционной системой, например, для управления файловой системой или переменными окружения
import pandas as pd  # Библиотека pandas для работы с данными в табличном формате, в частности с DataFrame
import getpass  # Модуль для безопасного ввода пароля в терминале
import openai  # Библиотека для взаимодействия с API OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings  # Импорт класса OpenAIEmbeddings для создания эмбеддингов на основе OpenAI
from langchain.vectorstores import FAISS  # Импорт FAISS для работы с векторными базами данных
#from typing import List  # Импорт модуля List из библиотеки typing для аннотации типов, особенно списков
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Импорт класса для рекурсивного разбиения текста на части
import tiktoken # Модуль для работы с токенизацией текста
import re  # Модуль регулярных выражений для обработки строк с использованием шаблонов
import requests  # Модуль для отправки HTTP-запросов, получения данных по URL
#import openai  # Повторный импорт библиотеки openai для взаимодействия с API OpenAI (возможно, для обновления или во избежание ошибок)
import gdown  # Модуль для загрузки файлов из Google Drive по их URL
import time # Импортируем стандартный модуль time, который предоставляет функции для работы с временем
from langchain.docstore.document import Document  # Импорт класса Document для работы с документами в langchain
import textwrap
from IPython.display import clear_output  # Импорт функции для очистки вывода в Jupyter Notebook
#from langchain.chat_models import ChatOpenAI  # Импорт модели ChatOpenAI для создания чат-ботов на основе OpenAI
#from langchain.schema import (AIMessage, HumanMessage, SystemMessage)  # Импорт классов для работы с различными типами сообщений в чатах langchain
#pd.set_option('display.max_colwidth', None)  # Настройка pandas для отображения всех символов в ячейках DataFrame, без сокращения длинных строк

# Функция getpass.getpass() запрашивает ввод пароля или ключа без отображения ввода на экране
openai_key = getpass.getpass("OpenAI API Key:")
# Установка ключа API в переменную окружения
os.environ["OPENAI_API_KEY"] = openai_key
# Установка ключа API для использования библиотекой openai
openai.api_key = openai_key

import faiss
import pickle
import gdown
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore

def load_faiss_index_from_drive(embeddings):
    # Скачиваем файлы из Google Drive
    gdown.download('https://drive.google.com/uc?id=1jxGqxcb0N31Cxk5aYSXgrVw6FGGoCiES', '/content/faiss_index.faiss', quiet=False)
    gdown.download('https://drive.google.com/uc?id=1SEjtg2P7ovMOIRTPrBshZUQeTUfDs3n7', '/content/faiss_metadata.pkl', quiet=False)

    # Загружаем индекс FAISS
    index = faiss.read_index('/content/faiss_index.faiss')

    # Загружаем метаданные
    with open('/content/faiss_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)

    # Восстанавливаем объект InMemoryDocstore из метаданных
    docstore = InMemoryDocstore(metadata['docstore'])

    # Восстанавливаем объект FAISS из индекса и метаданных
    db = FAISS(embedding_function=embeddings.embed_query, index=index, docstore=docstore, index_to_docstore_id=metadata['index_to_docstore_id'])

    return db

# Пример использования
embeddings = OpenAIEmbeddings()
db = load_faiss_index_from_drive(embeddings)

clear_output()  # Очистка вывода в ячейке Jupyter Notebook для улучшения читаемости и удобства работы

## Загрузка базы знаний

Ссылка на базу знаний:

https://docs.google.com/document/d/1aNcrmTAok2PCV5r6xPGswY8s7NKNlgC-mxyosEipjfo/edit?usp=sharing

In [ ]:
# Функция загрузки данных, которая принимает строку url в качестве аргумента и возвращает строку (текст документа)

def load_document_text_google(url: str) -> str:

    # Выполнение HTTP GET запроса для получения содержимого документа
    response = requests.get(url.replace('/edit?usp=sharing', '/export?format=txt'))

    # Проверка успешности запроса. Если запрос завершился с ошибкой (статусный код 4xx или 5xx), вызовется исключение, которое прервет выполнение функции
    response.raise_for_status()

    # Возвращение текста документа в качестве результата функции
    return response.text

In [ ]:
# Загружаем наш документ
text = load_document_text_google('https://docs.google.com/document/d/1aNcrmTAok2PCV5r6xPGswY8s7NKNlgC-mxyosEipjfo/edit?usp=sharing')

Содержание документа

In [ ]:
text

"\ufeff# Курсы английского языка онлайн под ваши цели\r\nПредоставим программу обучения, исходя из ваших запросов, поможем достичь цели, используя уроки по самым актуальным темам. \r\n\r\n\r\n## Что нужно, чтобы начать обучение?\r\n\r\n\r\nАбсолютно ничего, кроме устройства, у которого есть микрофон, камера и доступ к сайту Skyeng.\r\n\r\n\r\n## Не получается прийти на урок, что делать?\r\n\r\n\r\nНичего страшного, бывает! У учеников есть возможность отменить занятие за 8 часов до назначенного времени без потери оплаты. Так вы сможете переназначить его на удобное время, а у преподавателя останется возможность скорректировать расписание.\r\n\r\n\r\n## Как стать Преподавателем?\r\n\r\n\r\nОставьте заявку на сайте и запишите короткое аудиоинтервью о своем опыте. Мы свяжемся с вами по почте и расскажем о предстоящих этапах. Не переживайте: сначала мы познакомим вас с нашей платформой, затем методист сыграет роль ученика на вашем первом занятии. После успешного прохождения отбора вы сможете

## Деление на чанки

"Чанки" (от английского "chunks") - это термин, который в программировании и обработке данных обычно обозначает небольшие блоки данных. Так что основная цель использования чанков - это обработка больших объемов данных более эффективно и управляемо.

`RecursiveCharacterTextSplitter `- это класс в библиотеке langchain, который предназначен для разделения текста на фрагменты или "чанки".


> По умолчанию список разделителей включает в себя следующие символы: `['\n\n', '\n', ' ', '']`. Это означает, что сначала текст пытается разбиться по двум символам новой строки, затем по одному символу новой строки, затем по пробелу, и в конце, если ни один из предыдущих разделителей не сработал, текст разбивается на отдельные символы.

In [ ]:
# Функция разделения текста на чанки, которая принимает текст в виде строки, размер чанка и количество перекрывающихся символов между чанками

def split_text_into_chunks(text: str, chunk_size: int, chunk_overlap: int) -> str:

    # Инициализация пустого списка для хранения частей текста (чанков)
    source_chunks = []

    # Создание объекта класса RecursiveCharacterTextSplitter с заданным размером чанка и перекрытием для разбиения текста на части
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # Цикл для итерации по каждому чанкованному фрагменту текста
    for chunk in splitter.split_text(text):

        # Создание объекта Document для каждого чанка текста и добавление его в список source_chunks
        # page_content хранит текст чанка, а metadata (пока пустой словарь) может содержать дополнительную информацию
        source_chunks.append(Document(page_content=chunk, metadata={}))

    # Возвращение списка объектов Document, каждый из которых содержит часть исходного текста
    return source_chunks

Вызов функции

In [ ]:
# Задаем размер чанка
chunk_size = 1024

# Задаем размер перекрытия чанков
chunk_overlap = 0

# Вызываем функцию разеделения текста на чанки
source_chunks = split_text_into_chunks(text, chunk_size, chunk_overlap)

print("Общее количество чанков: ", len(source_chunks))
print("Пример чанка № 12:\n", source_chunks[11])

Общее количество чанков:  125
Пример чанка № 12:
 page_content='1. Дополнительные уроки в подарок\r\n2. 1 месяц разговорного клуба\r\n\r\n\r\n## Стоимость курсов с русскоязычным преподавателем\r\n\r\n\r\nВыучите новые слова на все случаи жизни, разберетесь даже в сложных временах и сломаете языковой барьер.\r\nот 950 ₽ за урок\r\n\r\n\r\n## Английский с англоязычным преподавателем\r\n\r\n\r\nНаучитесь понимать живую речь носителей и узнаете выражения, которые они используют каждый день.\r\nот 3 247 ₽ за урок\r\n\r\n\r\n## Премиум английский\r\n\r\n\r\nПремиум-преподаватели, гибкий график, персональный менеджер, индивидуальный план обучения. А также разговорный премиум-клуб и скрининг-тесты.\r\nот 1 290 ₽ за урок\r\n\r\n\r\n# Английский язык для взрослых онлайн\r\n\r\n\r\nПроходите индивидуальные уроки в удобное время из любой точки земного шара, от Москвы до Бангкока, и используйте английский уверенно в любой ситуации\r\n\r\n\r\n\r\n\r\n## Как проходит индивидуальное обучение в Skyeng\

## Эмбеддинги

>Эмбеддинги (или векторные представления) - это способ представления текстовых данных в виде числовых векторов, которые могут быть обработаны машинными алгоритмами

Создание эмбеддингов для чанок c помощью OpenAI

In [ ]:
embeddings = OpenAIEmbeddings()  # Создаем объект эмбеддингов от OpenAI

In [ ]:
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='dfghrilt4567erhgdfku5', openai_organization='', allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={})

## FAISS

>FAISS — это способ структурированного хранения эмбеддингов (векторов) и выполнения быстрого поиска по ним. FAISS позволяет легко и быстро находить семантически близкие векторы в большой базе данных векторов

Вспомогательная функция подсчета токенов

In [ ]:
# Функция для подсчета токенов при создании базы эмбеддингов,которая принимает текст и название кодировки, используемой для токенизации текста
def num_tokens_from_string(string: str, encoding_name: str) -> int:

    # Получаем объект кодировки по ее имени
    encoding = tiktoken.get_encoding(encoding_name)

    # Кодируем строку в токены и подсчитываем количество токенов
    num_tokens = len(encoding.encode(string))

    # Возвращаем количество токенов
    return num_tokens

Функция создание базы данных эмбедингов FAISS с помощью библиотеки langchain

In [ ]:
# Функция перевода чанков текста в векторы многомерного пространства
def create_faiss_index_from_chunks(source_chunks, embeddings):

    # Инициализируем счетчик токенов
    count_tokens = 0

    # Если документ не пуст, создаем и сохраняем базу индексов эмбеддингов фрагментов документа
    if len(source_chunks):

        # Начало отсчета времени
        start_time = time.time()

        # Создаем базу данных эмбеддингов
        db = FAISS.from_documents(source_chunks, embeddings)

        # Конец отсчета времени
        end_time = time.time()

        # Подсчет затраченного времени
        elapsed_time2 = end_time - start_time

        print(f"Время создания векторной базы: {elapsed_time2:.2f} секунд")

        # Подсчитываем количество токенов в документе
        count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
        print('Количество токенов в документе :', count_token)
        print('ЦЕНА запроса:', 0.0001 * (count_token / 1000), ' $')  # Вычисляем стоимость запроса

    return db

Вызов функции

In [ ]:
# Вызываем функцию перевода текста в векторы
db = create_faiss_index_from_chunks(source_chunks, embeddings)

Время создания векторной базы: 1.45 секунд
Количество токенов в документе : 52299
ЦЕНА запроса: 0.0052299  $


Результат работы:
> База данных FAISS

In [ ]:
db

In [ ]:
# Вывод информации об индексе FAISS
def print_faiss_index_info(db):
    index = db.index  # Получаем индекс FAISS
    print(f"Количество векторов в индексе: {index.ntotal}")
    print(f"Размерность векторов: {index.d}")

print_faiss_index_info(db)

Количество векторов в индексе: 125
Размерность векторов: 1536


In [ ]:
db.docstore._dict # словарь объекта хранилища документов

{'5c6627b4-22cf-48b1-8b19-6b4642fe83e4': Document(page_content='\ufeff# Курсы английского языка онлайн под ваши цели\r\nПредоставим программу обучения, исходя из ваших запросов, поможем достичь цели, используя уроки по самым актуальным темам. \r\n\r\n\r\n## Что нужно, чтобы начать обучение?\r\n\r\n\r\nАбсолютно ничего, кроме устройства, у которого есть микрофон, камера и доступ к сайту Skyeng.\r\n\r\n\r\n## Не получается прийти на урок, что делать?\r\n\r\n\r\nНичего страшного, бывает! У учеников есть возможность отменить занятие за 8 часов до назначенного времени без потери оплаты. Так вы сможете переназначить его на удобное время, а у преподавателя останется возможность скорректировать расписание.\r\n\r\n\r\n## Как стать Преподавателем?\r\n\r\n\r\nОставьте заявку на сайте и запишите короткое аудиоинтервью о своем опыте. Мы свяжемся с вами по почте и расскажем о предстоящих этапах. Не переживайте: сначала мы познакомим вас с нашей платформой, затем методист сыграет роль ученика на ваше

Пример вектора эмбединга

In [ ]:
# Получаем индекс FAISS из объекта db
# Поле index содержит сам индекс FAISS, который используется для хранения и поиска векторов
index = db.index

# Извлекаем векторы из индекса
# Метод reconstruct_n(start, count) извлекает count векторов, начиная с позиции start
# В данном случае, start = 0 и count = index.ntotal, что означает извлечение всех векторов из индекса
vectors = index.reconstruct_n(0, index.ntotal)

print("Извлеченные векторы:")
print(vectors)

Извлеченные векторы:
[[-0.0053272  -0.00761123 -0.01140048 ... -0.0023511   0.00261943
  -0.02162295]
 [-0.00168443  0.00259457 -0.01246051 ... -0.00478318  0.00445252
  -0.02543174]
 [-0.00724445  0.00864432  0.01082475 ... -0.00098136  0.00863142
  -0.04900167]
 ...
 [-0.00077495  0.0106073   0.0114803  ...  0.01189383 -0.00289305
  -0.04382076]
 [-0.01318105  0.01266658  0.01172228 ... -0.00130736  0.0020514
  -0.02967039]
 [-0.01194039 -0.00646881  0.01079126 ...  0.0018079   0.01459528
  -0.0295604 ]]


## Поиск по базе

Метод `similarity_search_with_score` - используется для поиска похожих запросу чанков по близости векторов в многомерном пространстве. Пи этом он возвращает оценку схожести документов

In [ ]:
topic = "как у вас происходит обучение?" # вопрос
k = 3 # количество наиболее релевантных чанков

In [ ]:
# Поиск документов, похожих на заданную тему/вопрос
docs_and_scores = db.similarity_search_with_score(topic, k=k)

# Создание содержимого сообщения, объединяя текст найденных документов и добавляя оценку схожести
message_parts = []  # Инициализируем список для хранения частей сообщения
for doc, score in docs_and_scores:
    # Форматируем текст документа с оценкой схожести
    doc_text_with_similarity = f'\n=====================\nОценка схожести: {score:.4f}\n{doc.page_content}\n'
    # Добавляем отформатированный текст в список частей сообщения
    message_parts.append(doc_text_with_similarity)

# Объединяем все части сообщения в одну строку и заменяем все вхождения '\r\n' на пробелы
message_content = re.sub(r'\r\n', ' ', '\n '.join(message_parts))

# Печать чанков и их оценок схожести
print('Чанки:\n ======================================== \n', message_content)

Чанки:
 
Оценка схожести: 0.2445
У каждого свой языковой уровень, темп и удобный ему формат обучения. Мы подбираем план обучения под потребности и цели каждого ученика и, самое главное, доводим до результата с помощью проверенной коммуникативной методологии и системы контроля прогресса. Нам важно, чтобы вы начали использовать язык уже с первого занятия!   ## Как мы достигаем эффективного обучения?   1. Наши курсы разработаны лучшими методистами школы с длительным опытом преподавания онлайн и завалидированы иностранными экспертами. 2. Эмоциональный интеллект, теория лени, гастротуризм, цифровой след, рейтинг городов — никаких школьных топиков. Наши уроки построены на темах, о которых говорит весь мир, с упором на использование в реальной жизни. 3. Мы строим учебный план, основываясь на ваших интересах и целях. Мы подбираем самые подходящие методы обучения в зависимости от ваших предпочтений и навыков.

 
Оценка схожести: 0.2756
## Как проходят занятия на платформе   1. Не нужно ничего с

## Запрос

ChatGPT использует идею "ролей". Каждое сообщение в списке сообщений - это набор данных, включающий два элемента: role и content. Role может быть одной из двух вещей: "system", "user", указывая, кто в данный момент "разговаривает". Content - это сам текст сообщения.

Обычно, в диалоге участвуют две главные роли:

"system": Роль системы используется для задания общей обстановки разговора. Это своего рода руководство для модели, указывающее, как она должна вести себя в процессе диалога.

"user": Это роль человека, который ведет диалог с моделью. Ваши вопросы модели будут представлены под этой ролью. Также сюда мы можем подавать определенные инструкции для модели (самые основные и важные).

In [ ]:
# Подгружаем промт
system = ''' # Ты профессиональный помощник в чате компании


## 1. Общие обязанности и цели - Компания продает курсы по английскому языку. У компании есть большой документ со всеми материалами
о продуктах компании. - Твоя обязанность: Дать Клиенту краткий корректный ответ на русском языке в чате, опираясь на отрывки
из этого документа. - Твоя цель: Отвечать максимально кратко и точно по документу, не придумывать ничего от себя.


## 2. Ограничения в общении - Запрещено общаться на стороннюю тему. Если Клиент задает стороннюю тему, спрашивает не
по теме английского языка, не по материалам и продуктам Компании, ты категорически отказываешься
отвечать.


## 3. Секретность документа
- Запрещено упоминать в ответе, что ты анализировал отрывки документов и брал оттуда информацию.

'''

# Инструкция для модели о том, как использовать контекст для ответа на вопрос
instruction = '''Проанализируй предыдущий диалог чтобы написать свой ответ последовательным и логичным.
Категорически запрещено повторяться и здороваться.
Используйте следующие фрагменты контекста, чтобы ответить на вопрос в конце.
Стилистика ответа должна быть поддерживающей беседу в контексте важности и полезности изучения английского языка.
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумывать ответ.
Тебе запрещено продавать, предлагать курсы. Запрещено спрашивать клиента что его еще интересует.'''

Функция для получения ответа на вопрос на основе заданной темы и поискового индекса

In [ ]:
def answer_index(system, instruction, topic, search_index, temp=1, verbose=0, k=3):
    """
    Параметры:
    system (str): Промт.
    instruction (str): Инструкция для пользователя.
    topic (str): Тема или вопрос, на который нужно ответить.
    search_index (object): Векторная база.
    temp (float): Температура для генерации ответов модели (по умолчанию 1).
    verbose (int): Флаг для включения/выключения подробного вывода (по умолчанию 0).
    k (int): Количество похожих документов для поиска (по умолчанию 3).
    """

    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)

    # Если включен подробный вывод, напечатать разделитель
    if verbose: print('\n ========\n')

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nОтвет:"}
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI
    completion = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,  # Сообщения для модели
        temperature=temp
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Если включен подробный вывод, напечатать разделитель
    if verbose: print('\n =========================================== ')

    # Получение ответа от модели
    answer = completion.choices[0].message.content
    print()
    print()
    print("Полный ответ от GPT:")
    print(completion)
    print()
    print("Ответ:")
    print(answer)
    print()
    print(f"Количество использованных токенов на запрос: {completion['usage']['prompt_tokens']}")
    print(f"Количество использованных токенов на ответ: {completion['usage']['completion_tokens']}")
    print(f"Общее количество использованных токенов (вопрос-ответ): {completion['usage']['total_tokens']}")
    print()
    print('ЦЕНА запроса:', (5 * completion['usage']['prompt_tokens'] / 1000000) + (15 * completion['usage']['completion_tokens']/ 1000000), ' $')  # Вычисляем стоимость запроса
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

Параметры для запроса

In [ ]:
# Задаем импровизацию ответа, 0 - ответ только на основании текста наших документов
temperature = 0

# Задаем количество чанков для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose = 1

Запрос

In [ ]:
question = "Какова продолжительность урока английского в Skyeng?"
result_df = answer_index(system, instruction, question, db, temperature, relevant_chanks)



Чанки :
 
--------------------
Кстати, если вы хотите отдохнуть от занятий или временно не можете приходить на уроки, в личном кабинете можно оформить отпуск. Он на время заморозит ваш прогресс. После вы сможете вернуться к занятиям, когда будете готовы. Мы подождем, и уроки не сгорят.   ## Сколько длится один урок? Одно занятие в Skyeng длится 50 минут.   ## Кто будет вести курс?   Вы будете заниматься на курсе английского языка с одним из наших учителей — опытных педагогов. Выбрать учителя можно самому — почитайте о них больше на странице «Преподаватели» и начните учиться с тем, кто вам ближе.   Есть и второй вариант. Расскажите нам о том, что вы цените в учителе, и мы сами найдем подходящего по вашим критериям. А если что-то пойдет не так, его можно будет заменить. Это легко сделать в личном кабинете.   ## Что еще входит в цену курса, кроме самого курса?

 
--------------------
Кстати, если вы хотите отдохнуть от занятий или временно не можете приходить на уроки, в личном кабинете

Запрос с добавлением истории диалога

In [ ]:
def answer_kons_d(system, instruction, topic, search_index, summary_history, temp=1, verbose=0, k=3):
    """
    Параметры:
    system (str): Промт.
    instruction (str): Инструкция для пользователя.
    topic (str): Тема или вопрос, на который нужно ответить.
    search_index (object): Векторная база.
    summary_history (str): История предыдущих сообщений диалога.
    temp (float): Температура для генерации ответов модели (по умолчанию 1).
    verbose (int): Флаг для включения/выключения подробного вывода (по умолчанию 0).
    k (int): Количество похожих документов для поиска (по умолчанию 8).
    """

    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nХронология предыдущих сообщений диалога: {summary_history}\n\nОтвет:"}
    ]

    # Создание запроса к модели OpenAI
    completion = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,  # Сообщения для модели
        temperature=temp
    )

    # Получение ответа от модели
    answer = completion.choices[0].message.content

    print('\n==================')
    print("Менеджер:")
    # Форматирование ответа для удобства чтения
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(formatted_answer)

    return answer

Параметры для запроса

In [ ]:
# Задаем импровизацию ответа, 0 - ответ только на основании текста наших документов
temperature = 0

# Задаем количество чанков для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose = 0

Запрос без истории

In [ ]:
# Вопрос
topic = "А если два?"

# История диалога
history = [""]

# Вызов функции answer_kons_d с корректным именем переменной для хранения результата
answer_response = answer_kons_d(system, instruction, topic, db, history, temperature, verbose, relevant_chanks)
print(answer_response)


Менеджер:
Одно занятие в Skyeng длится 50 минут.


Запрос с историей

In [ ]:
# Вопрос
topic = "А если два?"

# История диалога
history = ["Сколько раз в неделю необходимо заниматься?", "Для достижения наилучших результатов мы рекомендуем заниматься 3 раза в неделю."]

# Вызов функции answer_kons_d с корректным именем переменной для хранения результата
answer_response = answer_kons_d(system, instruction, topic, db, history, temperature, verbose, relevant_chanks)
print(answer_response)


Менеджер:
Если два раза в неделю, прогресс будет медленнее, но вы все равно сможете достичь своих целей. Главное — регулярность и мотивация.


## Диалог

In [ ]:
# Подгружаем промты
system = '''
Ты профессиональный помощник в чате компании.

## 1. Общие обязанности и цели
- Компания продает курсы по английскому языку. У компании есть большой документ со всеми материалами о продуктах компании.
- Твоя обязанность: Дать Клиенту краткий корректный ответ на русском языке в чате, опираясь на отрывки из этого документа.
- Твоя цель: Отвечать максимально кратко и точно по документу, не придумывать ничего от себя.

## 2. Ограничения в общении
- Запрещено общаться на стороннюю тему. Если Клиент задает вопрос не по теме английского языка, не по материалам и продуктам Компании, ты категорически отказываешься отвечать.

## 3. Секретность документа
- Запрещено упоминать в ответе, что ты анализировал отрывки документов и брал оттуда информацию.
'''

instruction = '''
Проанализируй предыдущий диалог чтобы написать свой ответ последовательным и логичным.
Категорически запрещено повторяться и здороваться.
Используй следующие фрагменты контекста, чтобы ответить на вопрос в конце.
Стилистика ответа должна быть поддерживающей беседу в контексте важности и полезности изучения английского языка.
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумывать ответ.
Тебе запрещено продавать, предлагать курсы. Запрещено спрашивать клиента что его еще интересует.
'''

# Функция для генерации ответа на вопрос пользователя на основе всей истории диалога
def answer_kons_d(system, instruction, summary_history, search_index):
    # Поиск документов, похожих на заданную тему/вопрос на основе всей истории диалога
    docs = search_index.similarity_search(summary_history, k=3)

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for doc in docs]))

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nИстория диалога:\n{summary_history}\n\nОтвет:"}
    ]

    # Создание запроса к модели OpenAI
    completion = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )

    # Получение ответа от модели
    answer = completion.choices[0].message.content
    return answer

# Функция для генерации следующего вопроса на основе всей истории диалога
def generate_follow_up_question(summary_history, search_index):
    follow_up_system = '''
    Ты профессиональный помощник в чате компании.
    '''
    follow_up_instruction = '''
    Проанализируй предыдущую историю диалога, чтобы задать один дополнительный вопрос на основании всей истории.
    Больше ничего не пиши. Укажи этот вопрос явно и лаконично.
    '''

    # Генерация следующего вопроса на основе истории диалога
    follow_up_question = answer_kons_d(
        system=follow_up_system,
        instruction=follow_up_instruction,
        summary_history=summary_history,
        search_index=search_index
    )

    return follow_up_question

# Основная функция для запуска бота и обработки диалога с пользователем
def run_bot():
    history_chat = []
    search_index = db  # Подгружаем индекс FAISS

    # Стандартный первый вопрос
    initial_question = "Здравствуйте! Какой у вас вопрос по курсам английского языка?"
    print(f"БОТ: {initial_question}")
    history_chat.append(f"БОТ: {initial_question}")

    while True:
        client_question = input("Пользователь: ").strip()
        if client_question.lower() in ['stop', 'стоп']:
            print("БОТ: Завершаю диалог. Всего доброго!")
            break

        # Сохраняем вопрос клиента в истории
        history_chat.append(f"Пользователь: {client_question}")

        # Генерация ответа на вопрос
        answer = answer_kons_d(
            system=system,
            instruction=instruction,
            summary_history='\n'.join(history_chat),
            search_index=search_index
        )

        # Вывод и сохранение ответа бота
        print(f"БОТ: {answer}")
        history_chat.append(f"БОТ: {answer}")

        # Генерация нового вопроса на основе всей истории диалога
        follow_up_question = generate_follow_up_question(
            summary_history='\n'.join(history_chat),
            search_index=search_index
        )

        # Вывод и сохранение нового вопроса
        print(f"БОТ: {follow_up_question}")
        history_chat.append(f"БОТ: {follow_up_question}")

# Запуск бота
run_bot()

БОТ: Здравствуйте! Какой у вас вопрос по курсам английского языка?
Пользователь: С чего начать изучени английского?
БОТ: Начните изучать английский с нуля, расширяя словарный запас, разбираясь в грамматике и учась понимать живую речь на слух. Это поможет свободно общаться, читать и писать на английском. Программа обучения включает 44 онлайн-урока и 9 тематических модулей, начиная с основ, таких как представление себя и знакомство с другими людьми.
БОТ: Когда вам будет удобно записаться на бесплатный вводный урок?
Пользователь: Давайте на завтра
БОТ: Отлично! Завтра мы сможем познакомить вас с нашей онлайн-платформой и помочь найти цель и подходящего учителя. Вы сможете начать понимать иностранную речь, читать, грамотно писать и бегло разговаривать.
БОТ: Какое время завтра вам будет удобно для вводного урока?
Пользователь: 20:00
БОТ: Завтра в 20:00 мы проведем для вас вводный урок. Вы познакомитесь с нашей онлайн-платформой, и мы поможем вам найти цель и подходящего учителя.
БОТ: Хотите